Load the packages to use in the project


In [2]:
library(dplyr)
library(magrittr)
library(flipTime) # To install it first install devtools, library(devtools) to load the it. 
# run install_github("Displayr/flipTime")
library(caret)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: ggplot2


Loading the data

In [3]:
phq9_data <- read.csv("data/PHQ_9 FORM.csv")


knowning what the data contains

In [5]:
str(phq9_data)


'data.frame':	14697 obs. of  15 variables:
 $ PHQ_9_Administration              : Factor w/ 175 levels "2017-01-03","2017-01-23",..: 110 116 114 75 43 114 45 77 116 45 ...
 $ Received_At                       : Factor w/ 7790 levels "2018-03-22 18:53:55 +00:00",..: 1876 82 4462 933 934 1232 2895 2897 2900 2896 ...
 $ Session_Number                    : Factor w/ 3 levels "Session 1","Session 12",..: 2 1 2 3 1 2 1 3 2 1 ...
 $ Meeting_Day                       : Factor w/ 8 levels "","Friday","Monday",..: 6 3 7 7 7 7 6 6 6 6 ...
 $ No_pleasure_doing_anything        : int  0 0 1 1 2 1 2 1 0 3 ...
 $ Feeling_sad__depressed_or_hopeless: int  0 0 0 0 1 0 3 0 0 2 ...
 $ Trouble_sleeping                  : int  0 0 0 1 1 1 2 1 0 2 ...
 $ Feeling_tired_or_little_energy    : int  1 1 0 1 2 0 1 0 0 2 ...
 $ Poor_appetite_or_over_eating      : int  1 1 1 1 2 1 2 2 1 1 ...
 $ self_guilt                        : int  0 1 0 1 1 0 2 0 0 2 ...
 $ Trouble_concentrating             : int  0 1 1 1 2 0 2 

In [6]:

head(phq9_data, 3)


PHQ_9_Administration,Received_At,Session_Number,Meeting_Day,No_pleasure_doing_anything,Feeling_sad__depressed_or_hopeless,Trouble_sleeping,Feeling_tired_or_little_energy,Poor_appetite_or_over_eating,self_guilt,Trouble_concentrating,Moving_or_speaking_slowly,Sucide_Thoughts,Total_Score,status
2018-04-12,2018-05-02 18:28:42 +00:00,Session 12,Thursday,0,0,0,1,1,0,0,0,0,2,Not Difficult at All
2018-04-19,2018-04-19 14:06:24 +00:00,Session 1,Monday,0,0,0,1,1,1,1,2,1,7,Somewhat Difficult
2018-04-17,2018-05-24 10:26:25 +00:00,Session 12,Tuesday,1,0,0,0,1,0,1,0,0,3,Not Difficult at All


In [7]:

tail(phq9_data, 3)

,PHQ_9_Administration,Received_At,Session_Number,Meeting_Day,No_pleasure_doing_anything,Feeling_sad__depressed_or_hopeless,Trouble_sleeping,Feeling_tired_or_little_energy,Poor_appetite_or_over_eating,self_guilt,Trouble_concentrating,Moving_or_speaking_slowly,Sucide_Thoughts,Total_Score,status
14695,2018-01-23,2018-05-25 10:40:29 +00:00,Session 1,Tuesday,1,2,2,2,1,1,1,1,1,12,Somewhat Difficult
14696,2018-01-23,2018-05-25 10:40:54 +00:00,Session 1,Tuesday,1,2,1,2,1,2,1,2,1,13,Somewhat Difficult
14697,2018-01-31,2018-06-01 06:04:34 +00:00,Session 1,Wednesday,2,2,1,1,1,1,1,2,0,11,Not Difficult at All


First lets remove the timestamp from the date in Recieved_At column since the PHQ Administration column doesnot have it.
Removing time zone using grab sub string method, we had no better way of doing it


In [4]:
phq9_data$Received_At <- gsub(x=phq9_data$Received_At,pattern=" +00:00",
                              replacement="",fixed=T)

Lets first convert our two dates columns to Date since they are factors as seen from str() above


In [5]:
phq9_data$Received_At <-AsDateTime(phq9_data$Received_At) # using the flipTime package
phq9_data$PHQ_9_Administration <-AsDateTime(phq9_data$PHQ_9_Administration)

Since all our time formats are in POSIXct, lets use as.Date trick to get only date for Recieved at and transform PHQ_Administration to Date


In [6]:
phq9_data$Received_At <-as.Date(phq9_data$Received_At)
phq9_data$PHQ_9_Administration <-as.Date(phq9_data$PHQ_9_Administration)

IF we just tranformed it directly minus transforming date(factor) to POSIXct it would return NA

Getting days between PHQ_Administration and Recieved_at

In [7]:
phq9_data$days_in_between <- phq9_data$Received_At- phq9_data$PHQ_9_Administration
head(phq9_data, 3)

PHQ_9_Administration,Received_At,Session_Number,Meeting_Day,No_pleasure_doing_anything,Feeling_sad__depressed_or_hopeless,Trouble_sleeping,Feeling_tired_or_little_energy,Poor_appetite_or_over_eating,self_guilt,Trouble_concentrating,Moving_or_speaking_slowly,Sucide_Thoughts,Total_Score,status,days_in_between
2018-04-12,2018-05-02,Session 12,Thursday,0,0,0,1,1,0,0,0,0,2,Not Difficult at All,20 days
2018-04-19,2018-04-19,Session 1,Monday,0,0,0,1,1,1,1,2,1,7,Somewhat Difficult,0 days
2018-04-17,2018-05-24,Session 12,Tuesday,1,0,0,0,1,0,1,0,0,3,Not Difficult at All,37 days


We nolonger need the Received_At and PHQ_9_Administration columns.

In [8]:
phq9_data <- phq9_data[,3:16]
head(phq9_data,3)

Session_Number,Meeting_Day,No_pleasure_doing_anything,Feeling_sad__depressed_or_hopeless,Trouble_sleeping,Feeling_tired_or_little_energy,Poor_appetite_or_over_eating,self_guilt,Trouble_concentrating,Moving_or_speaking_slowly,Sucide_Thoughts,Total_Score,status,days_in_between
Session 12,Thursday,0,0,0,1,1,0,0,0,0,2,Not Difficult at All,20 days
Session 1,Monday,0,0,0,1,1,1,1,2,1,7,Somewhat Difficult,0 days
Session 12,Tuesday,1,0,0,0,1,0,1,0,0,3,Not Difficult at All,37 days


changing status column as character not factor so that its easy for us to play with it.

In [9]:
phq9_data$status <- as.character(phq9_data$status)


so since classification takes two factors lets transform status into two levels, 1 - difficuilt(extremely difficult and very difficult), 0 - (somewhat difficult and Not difficult at all)

In [10]:
phq9_data$status <- sub("Somewhat Difficult","Not Difficult at All",phq9_data[,13])
phq9_data$status <- sub("Very Difficult","Extremely Difficult",phq9_data[,13])


We tried it with a for loop but it took us 7.26 - 8 mins to finish running

we take back the status column to factors

In [11]:
phq9_data$status <- as.factor(phq9_data$status)

str(phq9_data$status)

 Factor w/ 2 levels "Extremely Difficult",..: 2 2 2 2 2 2 2 2 2 2 ...


Ready to start with svm


Many books, tutorials and lectures use the following approach:

1. Transform data to the format of an SVM package
2. Randomly try a few kernels and parameters
3. Test

lets see that!


Dividing our data into train and test set.

In [12]:
set.seed(2292018) # Random splitting of data, the set.seed is for all of us in different machines to get the same values as train and test set.
Train <-createDataPartition(phq9_data$status, p =0.8, list = FALSE)

trainset <-phq9_data[Train,]
testset <-phq9_data[-Train,]

dim(trainset)
dim(testset)

[1] 11758    14

[1] 2939   14

Fitting the model using a linear kernal

In [17]:
control_trn1 <- trainControl(method = "repeatedcv", number = 4, repeats = 2)
set.seed(2018)
 
Linear_svm <- train(status ~., data = phq9_data, method = "svmLinear",
                 trControl= control_trn1
                 )
Linear_svm


Support Vector Machines with Linear Kernel 

14697 samples
   13 predictor
    2 classes: 'Extremely Difficult', 'Not Difficult at All' 

No pre-processing
Resampling: Cross-Validated (4 fold, repeated 2 times) 
Summary of sample sizes: 11022, 11023, 11024, 11022, 11023, 11023, ... 
Resampling results:

  Accuracy  Kappa    
  0.829387  0.4931397

Tuning parameter 'C' was held constant at a value of 1

Methos for resampling include: Bootstrap, k-fold Cross Validation, Repeated k-fold Cross Validation, and Leave One Out Cross Validation

In [18]:
# Making prediction and evaluating our model

predictions = predict(Linear_svm, newdata = testset)

cm = confusionMatrix(predictions, testset$status)
cm

Confusion Matrix and Statistics

                      Reference
Prediction             Extremely Difficult Not Difficult at All
  Extremely Difficult                  374                  197
  Not Difficult at All                 316                 2052
                                             
               Accuracy : 0.8255             
                 95% CI : (0.8112, 0.839)    
    No Information Rate : 0.7652             
    P-Value [Acc > NIR] : 1.024e-15          
                                             
                  Kappa : 0.4833             
 Mcnemar's Test P-Value : 1.890e-07          
                                             
            Sensitivity : 0.5420             
            Specificity : 0.9124             
         Pos Pred Value : 0.6550             
         Neg Pred Value : 0.8666             
             Prevalence : 0.2348             
         Detection Rate : 0.1273             
   Detection Prevalence : 0.1943             
      B

Lets try another kernal function

In [19]:
control_trn2 <- trainControl(method = "repeatedcv", number = 4, repeats = 2)
set.seed(2019)
 
Radial_svm <- train(status ~., data = phq9_data, method = "svmRadial",
                 trControl= control_trn2
                 )
Radial_svm


Support Vector Machines with Radial Basis Function Kernel 

14697 samples
   13 predictor
    2 classes: 'Extremely Difficult', 'Not Difficult at All' 

No pre-processing
Resampling: Cross-Validated (4 fold, repeated 2 times) 
Summary of sample sizes: 11023, 11023, 11022, 11023, 11023, 11022, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.8301356  0.4879189
  0.50  0.8322788  0.4975750
  1.00  0.8329932  0.5016253

Tuning parameter 'sigma' was held constant at a value of 0.04353732
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.04353732 and C = 1.

Predictions

In [20]:
# Making prediction and evaluating our model

predictions = predict(Radial_svm, newdata = testset)

cm = confusionMatrix(predictions, testset$status)
cm


Confusion Matrix and Statistics

                      Reference
Prediction             Extremely Difficult Not Difficult at All
  Extremely Difficult                  396                  163
  Not Difficult at All                 294                 2086
                                             
               Accuracy : 0.8445             
                 95% CI : (0.8309, 0.8574)   
    No Information Rate : 0.7652             
    P-Value [Acc > NIR] : < 2.2e-16          
                                             
                  Kappa : 0.5368             
 Mcnemar's Test P-Value : 1.193e-09          
                                             
            Sensitivity : 0.5739             
            Specificity : 0.9275             
         Pos Pred Value : 0.7084             
         Neg Pred Value : 0.8765             
             Prevalence : 0.2348             
         Detection Rate : 0.1347             
   Detection Prevalence : 0.1902             
      B


Useful link for different kernal to try

[Link](https://rdrr.io/cran/caret/man/models.html)


The above approach is what many people use but it is not appropriate. this is the best proposed way to handle classification tasks using svm, it works well if reasonsable accuracy is your concern.

1. Transform data to the format of an SVM package
2. Conduct feature selection
3. perform feature scaling
4. Consider the RBF kernel K(x, y) = e −γkx−yk
5. Use cross-validation to find the best parameter C and γ
6. Use the best parameter C and γ to train the whole training set 


Lets see that in action!

First we have already tranformed the data to an SVM package format

Feature selection - to keep it short we shall leave it out, backward elimination would be better.



In [21]:
control_trn3 <- trainControl(method = "repeatedcv", number = 4, repeats = 2)
set.seed(2020)
 
svm_better1 <- train(status ~., data = phq9_data, method = "svmRadial",
                 trControl= control_trn3,
                 preProcess = c("center", "scale"),
                 tuneLength = 5)
svm_better1

Support Vector Machines with Radial Basis Function Kernel 

14697 samples
   13 predictor
    2 classes: 'Extremely Difficult', 'Not Difficult at All' 

Pre-processing: centered (20), scaled (20) 
Resampling: Cross-Validated (4 fold, repeated 2 times) 
Summary of sample sizes: 11022, 11022, 11023, 11024, 11022, 11023, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.8299996  0.4882029
  0.50  0.8323811  0.4992513
  1.00  0.8342861  0.5064041
  2.00  0.8346605  0.5085457
  4.00  0.8352391  0.5104324

Tuning parameter 'sigma' was held constant at a value of 0.04376664
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.04376664 and C = 4.

Predictions

In [22]:
predictions = predict(svm_better1, newdata = testset)

cm = confusionMatrix(predictions, testset$status)
cm

Confusion Matrix and Statistics

                      Reference
Prediction             Extremely Difficult Not Difficult at All
  Extremely Difficult                  428                  145
  Not Difficult at All                 262                 2104
                                             
               Accuracy : 0.8615             
                 95% CI : (0.8485, 0.8738)   
    No Information Rate : 0.7652             
    P-Value [Acc > NIR] : < 2.2e-16          
                                             
                  Kappa : 0.5905             
 Mcnemar's Test P-Value : 8.929e-09          
                                             
            Sensitivity : 0.6203             
            Specificity : 0.9355             
         Pos Pred Value : 0.7469             
         Neg Pred Value : 0.8893             
             Prevalence : 0.2348             
         Detection Rate : 0.1456             
   Detection Prevalence : 0.1950             
      B

Also another way we could set the cost/ tunning parameters is by using the grid. that is

In [ ]:
control_trn4 <- trainControl(method = "repeatedcv", number = 4, repeats = 2)

grid_radial <- expand.grid(sigma = c(0.01, 0.05, 0.08, 0.1),
 C = c(0.1, 0.5, 1.0, 1.5))
set.seed(2021)
 
svm_better2 <- train(status ~., data = phq9_data, method = "svmRadial",
                 trControl= control_trn4,
                 preProcess = c("center", "scale"),
                 tuneGrid = grid_radial,
                 tuneLength = 5)
svm_better2

Predictions on the test set

In [ ]:
predictions = predict(svm_better2, newdata = testset)

cm = confusionMatrix(predictions, testset$status)
cm

#### Useful Resources to learn more

Theory 

SVM by Chih-Jen Lin available at Data Science Summer school. its video quality is very poor but good material

Introduction to Statistical Learning svm chapter

Support Vector Machine succintly by Alexandre Kowalczyk freely available at www.syncfusion.com

Practical 

Caret and e1071 Documentation